### Using machine learning to create a model to diagnosis diabetes


In [1]:
import numpy as np
import pandas as pd

#### Load dataset

In [2]:
df = pd.read_csv("./diabetes.csv")

#### The dataset has 9 columns, the Outcome is the target column

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,2,138,62,35,0,33.6,0.127,47,1
1,0,84,82,31,125,38.2,0.233,23,0
2,0,145,0,0,0,44.2,0.630,31,1
3,0,135,68,42,250,42.3,0.365,24,1
4,1,139,62,41,480,40.7,0.536,21,0


First, we need to check if the dataset has some Nan values and fill it with the data according with the column.

We can see below that there isn't columns with Nan Values

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               2000 non-null   int64  
 1   Glucose                   2000 non-null   int64  
 2   BloodPressure             2000 non-null   int64  
 3   SkinThickness             2000 non-null   int64  
 4   Insulin                   2000 non-null   int64  
 5   BMI                       2000 non-null   float64
 6   DiabetesPedigreeFunction  2000 non-null   float64
 7   Age                       2000 non-null   int64  
 8   Outcome                   2000 non-null   int64  
dtypes: float64(2), int64(7)
memory usage: 140.8 KB


All columns are numeric, so we'll need only to normalize the values

In [5]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,3.703500,121.182500,69.145500,20.935000,80.254000,32.193000,0.470930,33.090500,0.342000
std,3.306063,32.068636,19.188315,16.103243,111.180534,8.149901,0.323553,11.786423,0.474498
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,63.500000,0.000000,0.000000,27.375000,0.244000,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,40.000000,32.300000,0.376000,29.000000,0.000000
75%,6.000000,141.000000,80.000000,32.000000,130.000000,36.800000,0.624000,40.000000,1.000000
max,17.000000,199.000000,122.000000,110.000000,744.000000,80.600000,2.420000,81.000000,1.000000


Then we'll need to separate the dataset between train dataset and test dataset

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=["Outcome"]), df["Outcome"], test_size=0.3, random_state=42)

And normalize the TRAIN dataset

In [7]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_norm = pd.DataFrame(scaler.fit_transform(X_train))

In [8]:
X_train_norm

,0,1,2,3,4,5,6,7
0,0.352941,0.623116,0.590164,0.000000,0.000000,0.342432,0.123826,0.133333
1,0.058824,0.597990,0.360656,0.427273,0.084677,0.440447,0.086251,0.066667
2,0.470588,0.552764,0.622951,0.000000,0.000000,0.344913,0.067891,0.616667
3,0.000000,0.643216,0.557377,0.172727,0.241935,0.378412,0.560632,0.066667
4,0.000000,0.542714,0.557377,0.181818,0.000000,0.338710,0.302733,0.183333
...,...,...,...,...,...,...,...,...
1395,0.176471,0.618090,0.819672,0.318182,0.322581,0.710918,0.342442,0.016667
1396,0.000000,0.502513,0.573770,0.236364,0.067204,0.382134,0.221605,0.000000
1397,0.058824,0.502513,0.540984,0.263636,0.263441,0.397022,0.156277,0.350000
1398,0.235294,0.477387,0.524590,0.000000,0.000000,0.397022,0.035440,0.166667


#### The dataset is ready to be use*d* in a training process. Let's try to create a neural network using the lib Keras.
The warning messages can be ignored

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

neural_network = Sequential()
neural_network.add(Dense(units=16,activation='relu', input_shape = X_train_norm.iloc[0].shape))
neural_network.add(Dense(units=10,activation='relu'))
neural_network.add(Dense(units=2,activation='softmax'))

neural_network.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

This is how the network architecture should be like

In [10]:
from tensorflow.keras.utils import plot_model
plot_model(neural_network, show_shapes=True, show_layer_names=True)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


The array y_train need to be encoded with the values that represents the diagnosis

The index position representing the class will be set with 1.

[1, 0] - Class 0: negative diagnostic

[0, 1] - Class 1: positive diagnostic

In [11]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
y_train_encoded = encoder.fit_transform(y_train.values.reshape(-1, 1)).todense()
y_train_encoded

matrix([[0., 1.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [0., 1.],
        [1., 0.]])

Let's train the neural network

In [12]:
neural_network.fit(X_train_norm, y_train_encoded, epochs=100, batch_size=10)

Epoch 1/100
140/140 [==============================] - 1s 1ms/step - loss: 0.6653 - accuracy: 0.6650
Epoch 2/100
140/140 [==============================] - 0s 1ms/step - loss: 0.6416 - accuracy: 0.6621
Epoch 3/100
140/140 [==============================] - 0s 1ms/step - loss: 0.6208 - accuracy: 0.6650
Epoch 4/100
140/140 [==============================] - 0s 1ms/step - loss: 0.5987 - accuracy: 0.6793
Epoch 5/100
140/140 [==============================] - 0s 1ms/step - loss: 0.5777 - accuracy: 0.6950
Epoch 6/100
140/140 [==============================] - 0s 1ms/step - loss: 0.5593 - accuracy: 0.7093
Epoch 7/100
140/140 [==============================] - 0s 1ms/step - loss: 0.5463 - accuracy: 0.7121
Epoch 8/100
140/140 [==============================] - 0s 1ms/step - loss: 0.5331 - accuracy: 0.7286
Epoch 9/100
140/140 [==============================] - 0s 1ms/step - loss: 0.5213 - accuracy: 0.7336
Epoch 10/100
140/140 [==============================] - 0s 1ms/step - loss: 0.5141 - accura

140/140 [==============================] - 0s 1ms/step - loss: 0.4356 - accuracy: 0.7779
Epoch 82/100
140/140 [==============================] - 0s 1ms/step - loss: 0.4354 - accuracy: 0.7821
Epoch 83/100
140/140 [==============================] - 0s 1ms/step - loss: 0.4364 - accuracy: 0.7821
Epoch 84/100
140/140 [==============================] - 0s 1ms/step - loss: 0.4335 - accuracy: 0.7864
Epoch 85/100
140/140 [==============================] - 0s 1ms/step - loss: 0.4338 - accuracy: 0.7857
Epoch 86/100
140/140 [==============================] - 0s 1ms/step - loss: 0.4350 - accuracy: 0.7879
Epoch 87/100
140/140 [==============================] - 0s 1ms/step - loss: 0.4320 - accuracy: 0.7857
Epoch 88/100
140/140 [==============================] - 0s 1ms/step - loss: 0.4322 - accuracy: 0.7871
Epoch 89/100
140/140 [==============================] - 0s 1ms/step - loss: 0.4334 - accuracy: 0.7900
Epoch 90/100
140/140 [==============================] - 0s 1ms/step - loss: 0.4314 - accuracy: 

Now, we can test the accurracy achieved using the test dataset

In [13]:
X_test_norm = pd.DataFrame(scaler.transform(X_test))
predict = neural_network.predict(X_test_norm)

In [14]:
predict

array([[3.3870909e-01, 6.6129088e-01],
       [9.9937373e-01, 6.2633603e-04],
       [2.1825982e-02, 9.7817403e-01],
       ...,
       [7.2536439e-01, 2.7463558e-01],
       [1.3363432e-02, 9.8663658e-01],
       [9.3858087e-01, 6.1419114e-02]], dtype=float32)

y_pred will be contain the classes predict

In [15]:
y_pred = [np.argmax(x) for x in predict]

In [16]:
y_pred[:5]

[1, 0, 1, 1, 1]

Measuring the accuracy score

In [17]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.7916666666666666

#### This means that the neural network classifies the cases with 79% of accuracy 😃